<p style='text-align:center'>
PSY 394U <b>Methods for fMRI</b>, Fall 2019


<img style='width: 300px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2019/blob/master/Images/Placebo_Left.png?raw=true' alt='brain blobs'/>

</p>

<p style='text-align:center; font-size:40px; margin-bottom: 30px;'><b> Second and higher level analyses </b></p>

<p style='text-align:center; font-size:18px; margin-bottom: 32px;'><b> October 21 - 28, 2019</b></p>

<hr style='height:5px;border:none' />

# Second-level analysis, finger-foot-lips data
<hr style="height:1px;border:none" />

## Data
For this example, we will use the first-level analysis results from the finger-foot-lips task from the test-retest data set (ds114). The first-level analysis has been run for all the subjects, and their respective `.feat` directories have been generated. You can find them in the tar ball `ds114_BatchOutput_FingerFootLips.tar.gz`, available for download from [Box.com](https://utexas.box.com/s/3gjwyymbtkndnx85q0l3p83chgh0rwcz). 

Once you download the file, move it to the `ds114` directory on your local computer. Then you can unpack this tar ball (either locally on your computer, or via Docker) by running:
```
gunzip ds114_BatchOutput_FingerFootLips.tar.gz
```
Then
```
tar -xvf ds114_BatchOutput_FingerFootLips.tar
```
This should create a directory called `BatchOutput_FingerFootLips` under the `ds114` directory. 

## Analysis

As you recall from the previous class, there 7 different T contrasts estimated during the first-level analysis. We will focus on contrast 5 (finger > others, referred as cope5 hereafter). The goal of the second-level analysis is to see if all the subjects had the same activation patter. If the same brain area is activated across subjects, then we will be able to see it in the second-level analysis results.

First, we import necessary libraries, and define directories and parameters to be used in this analysis.

[`<Level2_FingerFootLips_Test_Cope5.py>`](https://github.com/sathayas/fMRIClassFall2019/blob/master/Level2/Level2_FingerFootLips_Test_Cope5.py)

In [ ]:
import os  # system functions
import nipype.interfaces.fsl as fsl  # fsl
from nipype import Node, Workflow  # components to construct workflow
from nipype import SelectFiles  # to facilitate file i/o
from nipype.interfaces.io import DataSink  # datasink


##### PARAMETERS #####
indCope = '5'  # the contrast of interest, finger vs others
indSes = 'test'  # the session of interest


##### DIRECTORY BUSINESS ######
# original data directory
dataDir = '/tmp/Data/ds114'
# Output directory
outDir = os.path.join(dataDir,'WorkflowOutput')

In preparation for the second-level analysis, we have to combine contrast images (cope images) and variance image (varcope images) across subjects. We do so by concatenating the `cope5.nii.gz` and `varcope5.nii.gz` images from all the subjects as 4D images, with subjects corresponding to the t-direction (or time points). In other words, 
   
   * time point 1: Subject 01
   * time point 2: Subject 02
   * ...
   * time point 10: Subject 10

This is done by the **`Merge`** tool from FSL. In addition to the `cope` and `varcope` images, we also combine mask images across subjects. Unfortunately the first-level analysis results are not stored in a BIDS-compliant file structure, you have to hard code to specify the appropriate directories and files.

In [ ]:
###########
#
# A LIST OF COPE, VARCOPE, AND MASK FILES TO BE MEREGED
#
###########
# directory where preprocessed fMRI data is located
baseDir = os.path.join(dataDir, 'BatchOutput_FingerFootLips/feat_dir')

# a list of subjects
subject_list = ['%02d' % i for i in range(1,11)]

listCopeFiles = []
listVarcopeFiles = []
listMaskFiles = []
for iSubj in subject_list:
    # full path to a cope image
    pathCope = os.path.join(baseDir,
                            'sub-' + iSubj,
                            'ses-' + indSes,
                            'run0.feat',
                            'stats',
                            'cope' + indCope + '.nii.gz')
    listCopeFiles.append(pathCope)

    # full path to a varcope image
    pathVarcope = os.path.join(baseDir,
                            'sub-' + iSubj,
                            'ses-' + indSes,
                            'run0.feat',
                            'stats',
                            'varcope' + indCope + '.nii.gz')
    listVarcopeFiles.append(pathVarcope)

    # full path to a mask image
    pathMask = os.path.join(baseDir,
                            'sub-' + iSubj,
                            'ses-' + indSes,
                            'run0.feat',
                            'mask.nii.gz')
    listMaskFiles.append(pathMask)

In [ ]:
###########
#
# NODES FOR THE MERGING IMAGES
#
###########
# merging cope files
copemerge = Node(fsl.Merge(dimension='t',
                           in_files=listCopeFiles),
                 name="copemerge")

# merging varcope files
varcopemerge = Node(fsl.Merge(dimension='t',
                           in_files=listVarcopeFiles),
                    name="varcopemerge")

# merging mask files
maskmerge = Node(fsl.Merge(dimension='t',
                           in_files=listMaskFiles),
                 name="maskmerge")

We are merging mask images so that we can include only the brain voxels in our second-level analysis. As you can imagine, different subjects have different voxels included in their brain mask.

<p style='text-align:center; font-size:18px;'>(Scroll through merged mask images)</p>

For the second-level analysis, we want to include voxels that are present in all subjects. So we need to create another mask image specifically for the second-level analysis, indicating only the voxels that are present in all subjects. This is done by generating the minimum image (across time-axis) of the merged mask image.

### Exercise
1. **Why minimum image?** Can you explain why the minimum image from the merged mask image contain only the voxels that are present in all subjects?  

<p style='text-align:center; font-size:18px;'>(Show the minimum mask image)</p>


In [ ]:
# calculating the minimum across time points on merged mask image
minmask = Node(fsl.MinImage(),
               name="minmask")

Now, we need to specify the regression model. In this analysis, we are simply doing a one-sample t-test, so the regression model should only include the intercept (or the mean). The regression model is specified by a dictionary of regressors. Each element in the dictionary is a regressor. They key corresponds to the name of the regressor, and the values contains a list of numbers, in the same order as the merged cope and varcope images.

In [ ]:
###########
#
# SETTING UP THE SECOND LEVEL ANALYSIS NODES
#
###########
# Dictionary with regressors
dictReg = {'reg1': [1]*len(subject_list) # vector of ones
          }


As for the contrast, since we only have one regressor, we can only do activation (i.e., \[1\]) and deactivation (i.e., \[-1\]).

In [ ]:
# Contrasts
cont01 = ['activation', 'T', list(dictReg.keys()), [1]]
cont02 = ['activation', 'T', list(dictReg.keys()), [-1]]

contrastList = [cont01, cont02]

Now we have the regression model (i.e., `dictReg`) and the contrasts (i.e., `contrastList`), we can set up the second level analysis. This is done by the **`MultipleRegressionModel`** tool in FSL. This produces necessary files for calculating various contrast and statistic images for the second-level analysis.

In [ ]:
# Setting up the second level analysis model node
level2design = Node(fsl.MultipleRegressDesign(contrasts=contrastList,
                                              regressors=dictReg),
                    name='level2design')

The actual calculation of statistic images is done by the **`FLAMEO`** tool in FSL. This step only requires a fixed-effect analysis (thus `run_mode='fe'`)

In [ ]:
# Model calculation by FLAMEO
flameo = Node(fsl.FLAMEO(run_mode='fe'),
              name="flameo")

Now let's create a datasink, and a workflow object

In [ ]:
# creating datasink to collect outputs
datasink = Node(DataSink(base_directory=
                         os.path.join(outDir,'FingerFootLips_Test_Cope5')),
                name='datasink')


###########
#
# SETTING UP THE WORKFLOW NODES
#
###########

# creating the workflow
secondLevel = Workflow(name="Level2", base_dir=outDir)

Now actually connecting nodes. First, we pass on various files describing the second-level analysis mode from `level2design` node to `flameo` node.

In [ ]:
# connecting nodes
secondLevel.connect(level2design, 'design_mat', flameo, 'design_file')
secondLevel.connect(level2design, 'design_con', flameo, 't_con_file')
secondLevel.connect(level2design, 'design_grp', flameo, 'cov_split_file')

Also, merged cope and varcope images are passed onto the `flameo` node.

In [ ]:
secondLevel.connect(copemerge, 'merged_file', flameo, 'cope_file')
secondLevel.connect(varcopemerge, 'merged_file', flameo, 'var_cope_file')

We calculate the minimum of the merged mask image, and pass onto `flameo`.

In [ ]:
secondLevel.connect(maskmerge, 'merged_file', minmask, 'in_file')
secondLevel.connect(minmask, 'out_file', flameo, 'mask_file')

And sending a directory of statistic images (`stats_dir`) to the datasink.

In [ ]:
secondLevel.connect(flameo, 'stats_dir', datasink, 'stats_dir')

And we run the workflow. It should run fairly quickly.

In [ ]:
# running the workflow
secondLevel.run()

## Results